In [12]:
from crewai import Task, Agent, Crew
from crewai_tools import JSONSearchTool

In [13]:
from src.utils import get_earnings_transcript
import re
import os
import json
from crewai_tools import JSONSearchTool

base_dir = "DATA"
os.makedirs(base_dir,exist_ok=True)
def clean_speakers(speaker):
    speaker = re.sub("\n", "", speaker)
    speaker = re.sub(":", "", speaker)
    return speaker

quarter = "Q1"
ticker = "AAPL"
year = "2023"

def get_save_earnings_data(quarter:str,ticker:str,year:str):
    ticker_path = os.path.join(base_dir,ticker)
    os.makedirs(ticker_path,exist_ok=True)
    ticker_year = os.path.join(ticker_path,year)
    os.makedirs(ticker_year,exist_ok=True)
    ticker_year_quarter = os.path.join(ticker_year,quarter)
    os.makedirs(ticker_year_quarter,exist_ok=True)

    resp_dict = get_earnings_transcript(quarter,ticker,year)

    content = resp_dict["content"]
    pattern = re.compile(r"\n(.*?):")
    matches = pattern.finditer(content)

    speakers_list = []
    ranges = []
    for match_ in matches:
        span_range = match_.span()
        ranges.append(span_range)
        speakers_list.append(match_.group())
    speakers_list = [clean_speakers(sl) for sl in speakers_list]

    earning_calls_docs = []
    for idx, speaker in enumerate(speakers_list[:-1]):
        start_range = ranges[idx][1]
        end_range = ranges[idx + 1][0]
        speaker_text = content[start_range + 1 : end_range]

        earning_calls_docs.append({"speaker":speaker,"quarter":quarter,"speaker_text":f"{speaker}: {speaker_text}"})

    earning_calls_docs.append({"speaker": speakers_list[-1], "quarter": quarter,'speaker_text':content[ranges[-1][1] :]})
    json_path = os.path.join(ticker_year_quarter,f'{quarter}.json')
    with open(json_path, 'w') as file:
        json.dump(earning_calls_docs, file, indent=4)
    json_tool = JSONSearchTool(json_path=json_path)
    return json_tool

In [17]:
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [18]:
from tenacity import RetryError
from crewai_tools import JSONSearchTool  # Updated import path

print("Earnings Call Q1")

def get_quarter_tool(quarters,ticker,year):
    earnings_calls_tools = []
    for quarter in quarters:
        try:
            curr_quarter_tool = get_save_earnings_data(quarter,ticker,year)
            earnings_calls_tools.append(curr_quarter_tool)
        except RetryError:
            print(f"Don't have the data for {quarter}")
    return earnings_calls_tools
earnings_calls_tools = get_quarter_tool(["Q1","Q2","Q3","Q4"],ticker,year)


Earnings Call Q1


Inserting batches in chromadb: 100%|██████████| 1/1 [00:20<00:00, 20.74s/it]


In [26]:
from src.secData import sec_main
sec_docs,sec_form_names = sec_main(ticker=ticker,year=year)

Started Scraping
Scraped
Started Extracting
Extracted


In [33]:
for sec_filings,sec_name in zip(sec_docs,sec_form_names):
    report_year = sec_filings[-2].split("-")[0]
    if sec_name.startswith("10-Q"):
        curr_quarter = sec_name.split("-")[-1]
        save_path = os.path.join(base_dir,ticker,report_year,curr_quarter)
    else:
        save_path = os.path.join(base_dir,ticker,report_year)
    os.makedirs(save_path,exist_ok=True)
    with open(os.path.join(save_path,f'{sec_name}.json'), 'w') as file:
        json.dump(sec_filings[-1], file, indent=4)

In [2]:
from src.vectorDatabaseDocker import create_database
ticker = "META"
year = "2023"
(
        qdrant_client,
        encoder,
        speakers_list_1,
        speakers_list_2,
        speakers_list_3,
        speakers_list_4,
        sec_form_names,
        earnings_call_quarter_vals,
    ) = create_database(ticker=ticker, year=int(float(year)))

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4
SEC
Started Scraping
Scraped
Started Extracting
Extracted


/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from src.queryDatabase import query_database_earnings_call,query_database_sec

In [9]:
from functools import partial
query_sec = partial(query_database_sec,qdrant_client=qdrant_client,encoder=encoder)
query_earnings_call = partial(query_database_earnings_call,qdrant_client=qdrant_client,encoder=encoder)

In [10]:
query_sec(question="AI training",search_form="10-Q1")

'RISK_FACTORS: other value to justify our investments, and our business may be adversely affected. We may not be successful in our artificial intelligence initiatives, which could adversely affect our business, reputation, or financial results. We are making significant investments in artificial intelligence (AI) initiatives, including to recommend relevant unconnected content across our products, enhance our advertising tools, and develop new product features using generative AI. In particular, we expect our AI. features using generative AI. In particular, we expect our AI initiatives will require increased investment in infrastructure and headcount. AI technologies are complex and rapidly evolving, and we face significant competition from other companies as well as an evolving regulatory landscape. These efforts, including the introduction of new products or changes to existing products, may result in new or enhanced governmental or regulatory scrutiny, litigation, ethical concerns, 

In [12]:
query_earnings_call(question="AI training",quarter="Q4",speakers_list=speakers_list_4)

"Mark Zuckerberg: Threads, although I'll note that Threads now has more people actively using it today than it did during its initial launch peak. So that one is, I think, on track to be a major success.  But normally, we learn and we iterate methodically. And we started doing that with our AI services in the fall launching Meta AI, our assistant; AI Studio, which the precursor to Creator AIs; our Alpha with Business AIs; and the Ray-Ban Meta smart glasses. And we've been tuning each of these, and we're getting closer tocrawl data set. And people share large numbers of public text posts and comments across our services as well.  But even more important in the upfront training corpus is the ability to establish the right feedback loops with hundreds of millions of people interacting with AI services across our products. And this feedback is a big part of how we've improved our AI systems so quickly with Reels and Ads, especially over the last couple of years when we had to re-architect 

In [13]:
sec_form_names

['10-K', '10-Q3', '10-Q2', '10-Q1']

In [14]:
earnings_call_quarter_vals

['Q1', 'Q2', 'Q3', 'Q4']

In [29]:
from crewai_tools import BaseTool, tool

sec_tools = []
for sec_form in sec_form_names:
    if "10-K" in sec_form:
        # description = 
        @tool("10-K data")
        def yearly_tool(question:str)->str:
            """
            This tool will fetch relevant documents from 10-K yearly filing to answer questions regarding the financial and other conditions about a public company 
            """
            return query_sec(question=question,search_form=sec_form)
        # sec_tools.update({sec_form:yearly_tool})
        sec_tools.append(yearly_tool)
    elif "10-Q" in sec_form:
        quarter = sec_form.split("-")[1]
        @tool(f"{sec_form} data")
        def quarterly_tool(question:str)->str:
            "This tool will fetch relevant documents from 10-Q quarterly filing from the quarter mentioned in the name of the tool." 
            "This tool will answer questions regarding the quarterly financial and other conditions about a public company" 
            return query_sec(question=question,search_form=sec_form)
        # sec_tools.update({sec_form:quarterly_tool})
        sec_tools.append(quarterly_tool)

In [30]:
sec_tools

[Tool(name='10-K data', description="10-K data(question: 'string') -              This tool will fetch relevant documents from 10-K yearly filing to answer questions regarding the financial and other conditions about a public company              ", args_schema=<class 'abc.10-Kdata'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7f6614a4dcf0>, func=<function yearly_tool at 0x7f653d7a81f0>),
 Tool(name='10-Q3 data', description="10-Q3 data(question: 'string') -              This tool will fetch relevant documents from 10-Q quarterly filing from the quarter mentioned in the name of the tool.              This tool will answer questions regarding the quarterly financial and other conditions about a public company              ", args_schema=<class 'abc.10-Q3Data'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7f6614a4dcf0>, func=<function quarterly_tool at 0x7f653d7a8310>),
 Tool(name='10-Q2 data', description="10-Q2 data(questio

In [33]:
earnings_calls_tools_list = []
for idx,speaker_list in enumerate([speakers_list_1,speakers_list_2,speakers_list_3,speakers_list_4]):
    if speaker_list == []: continue
    else:
        quarter = earnings_call_quarter_vals[idx]
        @tool(f"{quarter} earnings call data")
        def earnings_call_tool(question:str)->str:
            """
            This tool will fetch relevant documents from earnings call transcripts from the quarter mentioned in the name of the tool
            to answer questions regarding the financial and other conditions about a public company 
            """
            return query_earnings_call(question=question,quarter=quarter,speakers_list=speaker_list)
        # sec_tools.update({sec_form:yearly_tool})
        earnings_calls_tools_list.append(earnings_call_tool)

In [40]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [41]:
# Head tool --> direct question to one of the tools

router_financial_agent = Agent(
    role = "Financial data analyst",
    goal = "Analyze the question and determine which relevant financial document can "
            "best answer the question.",
    backstory = "With access to books about finance, SEC filings like 10-K for yearly "
                "data and 10-Qs for quarterly data, and earnings call transcripts quarterly "
                "data. With a understanding of the relevant data source, the Router Financial Agent "
                "will route the query to the relevant financial agent",
        verbose=True,
        allow_delegation=True
)

In [51]:
from langchain_openai import ChatOpenAI

sec_filings_agent = Agent(
    role = "SEC Filings Financial Data Analyst",
    goal = "Answer the question in detail by first using the relevant tool to get relevant documents ",
    backstory = "With access to 10-K yearly SEC filings and 10-Q quarterly SEC filings "
                "you can get relevant documents from these documents to answer the question. "
                "Don't answer the questions which is outside the scope of the relevant documents. "
                "Include all the relevant numerical figures in your answer.",
    verbose=True,
    allow_delegation=False,
    tools = sec_tools,
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
)

In [52]:
earnings_call_transcripts_agent  =Agent(
    role = "Earnings Call transcripts Financial Data Analyst",
    goal = "Answer the question in detail by first using the relevant tool from the relevant quarter to get relevant documents. ",
    backstory = "With access to earnings call transcripts quarterly data you can get relevant documents from these documents to answer the question. "
                "Don't answer the questions which is outside the scope of the relevant documents. "
                "Include all the relevant numerical figures in your answer.",
    verbose=True,
    allow_delegation=False,
    tools = earnings_calls_tools_list,
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125")        
)

In [53]:
routing_task = Task(
    description = (
        "Based on your understanding by thinking step by step "
        "about the question {question}, route the question to appropriate agent "
        "to best answer the question. You can delegate the question to an agent that can "
        "refer to SEC filings to answer the question, and other agent can refer to "
        "earnings call transcripts to answer the quetion. Based on your reasoning, route to relevant agent"
    ),
    expected_output = (
        "Information and reasoning to route the query to the relevant agent for answering the question"
    ),
    agent = router_financial_agent
)

In [54]:
sec_task = Task(
    description = (
        "Use one of the tool to get the relevant documents to the question {question}."
        "Using the relevant context only, answer in detail to the question, and don't skip out on numerical figures (if any)."
    ),
    expected_output = (
        "Detailed answer to the question, including the relevant context and numerical figures"
    ),
    tools = sec_tools,
    agent = sec_filings_agent
)

earnings_call_task = Task(
    description = (
        "Use one of the tool to get the relevant documents to the question {question}."
        "Using the relevant context only, answer in detail to the question, and don't skip out on numerical figures (if any)."
        "Also, mention the speakers from relevant context"
    ),
    expected_output = (
        "Detailed answer to the question, including the relevant context and numerical figures"
    ),
    tools = earnings_calls_tools_list,
    agent = earnings_call_transcripts_agent
)

In [55]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_docs_crew = Crew(
    agents=[sec_filings_agent, 
            earnings_call_transcripts_agent],
    
    tasks=[sec_task, 
           earnings_call_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.0),
    process=Process.hierarchical,
    verbose=True
)

2024-05-22 18:53:05,049 - 140077000359936 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [56]:
res = financial_docs_crew.kickoff({"question":"What did Mark Zuckerberg say about AI spending in the fourth quarter?"})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Use one of the tool to get the relevant documents to the question What did Mark Zuckerberg say about AI spending in the fourth quarter?.Using the relevant context only, answer in detail to the question, and don't skip out on numerical figures (if any).


> Entering new CrewAgentExecutor chain...
I need to use the tools available to get the relevant documents to answer the question about what Mark Zuckerberg said about AI spending in the fourth quarter. I must provide a detailed answer with all the necessary context and numerical figures.

Action:
Delegate work to co-worker

Action Input:
{
    "task": "Retrieve documents related to Mark Zuckerberg's statements on AI spending in the fourth quarter",
    "context": "Mark Zuckerberg is the CEO of Facebook and has been vocal about the company's investments in artificial intelligence. The fourth quarter refers to the last quarter of the fiscal year.",
    "coworker": "SEC Filings

In [57]:
res

'In the SEC Filings related to Facebook, Mark Zuckerberg mentioned that the company is making significant investments in artificial intelligence (AI) initiatives in the fourth quarter. These investments are aimed at recommending relevant unconnected content across their products, enhancing advertising tools, and developing new product features using generative AI. The management discussion highlighted that Facebook is responding to competitive pressures by investing in AI initiatives, including the development of an AI-powered discovery engine to recommend relevant unconnected content. This indicates that Mark Zuckerberg and Facebook are actively focusing on AI spending to stay competitive in the market and enhance user experience.'

In [62]:
res = financial_docs_crew.kickoff({"question":"What was the user growth in instagram and facebook?"})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Use one of the tool to get the relevant documents to the question What was the user growth in instagram and facebook?.Using the relevant context only, answer in detail to the question, and don't skip out on numerical figures (if any).


> Entering new CrewAgentExecutor chain...
I need to gather the relevant documents to answer the question about the user growth in Instagram and Facebook. I should use the tools available to delegate work or ask questions to my co-workers in order to get the necessary information.

Action:
Delegate work to co-worker

Action Input:
{
    "task": "Retrieve user growth data for Instagram and Facebook",
    "context": "I need detailed information on the user growth trends for Instagram and Facebook, including any recent data or reports available.",
    "co-worker": "SEC Filings Financial Data Analyst"
}


> Entering new CrewAgentExecutor chain...
I need to use the 10-K and 10-Q tools to retrieve t

In [64]:
print(res)

Based on the SEC Filings data, the user growth for Instagram and Facebook is as follows:

- Facebook daily active users (DAUs) were 2.04 billion on average for March 2023, an increase of 4% year-over-year.
- Facebook monthly active users (MAUs) were 2.99 billion as of March 31, 2023, an increase of 2% year-over-year.
- Family monthly active people (MAP) was 3.81 billion as of March 31, 2023, an increase of 5% year-over-year.
- Ad impressions delivered across the Family of Apps in the quarter of 2023 increased by 26% year-over-year.
- The average price per ad in the quarter of 2023 decreased by 17% year-over-year.

These figures indicate a positive user growth trend for both Instagram and Facebook, with an increase in active users and ad impressions. The decrease in the average price per ad may suggest a strategy to enhance ad engagement and reach a wider audience. The data was retrieved from the Management Discussion section of the 10-K filing for the relevant year.


In [65]:
res = financial_docs_crew.kickoff({"question":"How much did it cost to train foundational language models like Llama?"})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Use one of the tool to get the relevant documents to the question How much did it cost to train foundational language models like Llama?.Using the relevant context only, answer in detail to the question, and don't skip out on numerical figures (if any).


> Entering new CrewAgentExecutor chain...
I need to delegate the task of gathering the relevant documents to answer the question about the cost of training foundational language models like Llama.

Action:
Delegate work to co-worker

Action Input:
{
    "task": "Retrieve documents on the cost of training foundational language models like Llama",
    "context": "We need to provide a detailed answer to the question of how much it cost to train foundational language models like Llama. This information is crucial for our project analysis and decision-making process.",
    "coworker": "SEC Filings Financial Data Analyst"
}


> Entering new CrewAgentExecutor chain...
I need to us

In [66]:
print(res)

The cost of training foundational language models like Llama is not explicitly mentioned in the SEC filings or earnings call transcripts. While there is discussion about the increasing compute intensity required for training and operating future models, including Llama 3, there are no specific figures provided regarding the cost of training these models. The documents mainly focus on investments in AI initiatives, operational expenses related to data centers, and technical infrastructure, without detailing the specific cost related to training foundational language models like Llama.


In [67]:
res = financial_docs_crew.kickoff({"question":"What was capital expenditure the Meta Reality labs for Metaverse?"})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Use one of the tool to get the relevant documents to the question What was capital expenditure the Meta Reality labs for Metaverse?.Using the relevant context only, answer in detail to the question, and don't skip out on numerical figures (if any).


> Entering new CrewAgentExecutor chain...
I need to delegate the task of gathering the relevant documents to one of my co-workers in order to get the answer to the question about the capital expenditure of Meta Reality labs for Metaverse.

Action:
Delegate work to co-worker

Action Input:
{
    "task": "Gather relevant documents on capital expenditure of Meta Reality labs for Metaverse",
    "context": "We need to find out the capital expenditure of Meta Reality labs specifically for their work on the Metaverse project. This information is crucial for our analysis and decision-making process.",
    "coworker": "SEC Filings Financial Data Analyst"
}


> Entering new CrewAgentExec

In [68]:
print(res)

In the SEC Filings of Meta Reality Labs, it is mentioned that their investments in the metaverse efforts include expenses related to headcount, developing virtual and augmented reality devices, software for social platforms, neural interfaces, and other foundational technologies for the metaverse. The majority of their investments are focused on developing their family of apps, with 80% of their total costs and expenses recognized in their family of apps, and 20% recognized in Reality Labs. They are increasing investments in new platforms and technologies, including efforts related to building the metaverse. Some of these investments, particularly in Reality Labs, have generated limited revenue and reduced their operating margin and profitability.

The relevant documents from the SEC Filings indicate that Meta Reality Labs' investments in the metaverse include expenses related to headcount, technology development, and research and development for products for the metaverse. The majorit

## BOOKS

In [56]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
with open("books/Applied_corporate_finance/Applied_corporate_finance.md", "r", encoding="utf-8") as input_file:
    text = input_file.read()
# html = markdown.markdown(text)




In [57]:
# markdown_splitter.aggregate_lines_to_chunks(md_header_splits)

In [72]:
import os
book_dir = "BOOKS_MD"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,return_each_line=False)
all_docs = []
for book_name in os.listdir(book_dir):
    books_path = os.path.join(book_dir,book_name)
    with open(f"{books_path}/{book_name}.md", "r", encoding="utf-8") as input_file:
        text = input_file.read()
    md_header_splits = markdown_splitter.split_text(text)
    # print(md_header_splits)
    for md in md_header_splits:
        md.metadata.update({"book_name":book_name}) 
    # break
    all_docs.extend(md_header_splits)

In [77]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
chroma_db = Chroma.from_documents(all_docs, embedding_function, persist_directory="./chroma_db")

/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with y

In [78]:
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search("What is valuation?")